# W261 Final Project ETL for Development Sample

### Notebook Set-Up

In [1]:
# imports
import time
import numpy as np
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import DataFrame

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]
PWD

'/media/notebooks'

In [4]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "w261FinalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

__`REMINDER:`__ If you are running this notebook on the course docker container, you can monitor the progress of your jobs using the Spark UI at: http://localhost:4040/jobs/

## Load Data

In [5]:
original_trainRDD = sc.textFile('data/train.txt')
original_testRDD = sc.textFile('data/test.txt')

## Transformation

### Sample

In [6]:
#change the seed for a different sample
sampleRDD1, sampleRDD2 = original_trainRDD.randomSplit([0.99995,0.00005], seed = 1)
sampleRDD2.cache()

PythonRDD[4] at RDD at PythonRDD.scala:49

In [7]:
ncol = len(sampleRDD2.take(1)[0].split('\t'))
nrow = sampleRDD2.count()
print("This sample contains", str(nrow), "rows.")

This sample contains 2248 rows.


In [8]:
print("This sample contains", str(ncol), "columns.")

This sample contains 40 columns.


In [9]:
def avgFeatures(line):
    
    count = 0
    feats = line.split('\t')[1:]
    
    for feat in feats:
        if feat != '':
            count += 1

    return count

print("There is an average of", str(round(sampleRDD2.map(avgFeatures).mean(),2)), "populated features per observation.")

There is an average of 33.52 populated features per observation.


33.52179715302491

# Put in wide, sparse feature format

In [10]:
def parseCV(line):
    """
    Map record_csv_string --> (features, label)
    """

    # start of categorical features
    col_start = 14
    
    raw_values = line.split('\t')
    label = int(raw_values[0])
    
    # ignore numerics to start
    #numerical_values = list(pd.Series(raw_values[1:14]).apply(pd.to_numeric))
    numericals = []
    for idx, value in enumerate(raw_values[1:col_start]):
        if value != '':
            numericals.append('n' + str(idx) + '_' + str(value))
            
    
    categories = []
    for idx, value in enumerate(raw_values[col_start:]):
        if value != '':
            categories.append('c'+ str(idx) + '_' + str(value))

    return Row(label=label, raw=numericals + categories)


def vectorizeCV(DF):
    
    vectorizer = CountVectorizer()
    cv = CountVectorizer(inputCol="raw", outputCol="features", binary=True)
    
    model = cv.fit(DF)
    result = model.transform(DF)
    num_feats = cv.getVocabSize()
    
    return result


parsedDF = sampleRDD2.map(parseCV).toDF().cache()
vectorizedDF = vectorizeCV(parsedDF)

In [11]:
vectorizedDF.show(truncate=True)

+-----+--------------------+--------------------+
|label|                 raw|            features|
+-----+--------------------+--------------------+
|    0|[n1_4, n2_50, n3_...|(18545,[0,1,2,4,5...|
|    0|[n1_12, n2_20, n3...|(18545,[0,1,2,5,1...|
|    0|[n0_8, n1_17, n3_...|(18545,[0,1,4,12,...|
|    0|[n0_0, n1_144, n4...|(18545,[0,2,3,4,5...|
|    0|[n1_0, n2_5, n4_3...|(18545,[0,2,3,6,1...|
|    0|[n1_323, n2_2, n3...|(18545,[1,2,14,16...|
|    0|[n0_0, n1_424, n3...|(18545,[0,1,2,4,6...|
|    0|[n0_0, n1_13, n2_...|(18545,[0,1,2,5,6...|
|    0|[n1_180, n2_6, n3...|(18545,[1,2,8,14,...|
|    0|[n1_21, n2_3, n3_...|(18545,[1,2,6,10,...|
|    0|[n1_3, n2_7, n3_1...|(18545,[0,1,2,5,8...|
|    0|[n1_1, n2_2, n4_3...|(18545,[0,5,9,14,...|
|    0|[n0_0, n1_2, n4_3...|(18545,[0,2,5,6,1...|
|    1|[n1_-1, n4_26295,...|(18545,[0,2,5,9,2...|
|    0|[n0_0, n1_0, n2_1...|(18545,[0,1,5,7,9...|
|    1|[n0_1, n1_196, n3...|(18545,[0,1,2,3,5...|
|    0|[n0_0, n1_-1, n4_...|(18545,[0,1,3,4,5...|


In [12]:
vectorizedRDD = vectorizedDF.select(['label', 'features']).rdd.cache()
vectorizedRDD.take(1)

[Row(label=0, features=SparseVector(18545, {0: 1.0, 1: 1.0, 2: 1.0, 4: 1.0, 5: 1.0, 7: 1.0, 10: 1.0, 21: 1.0, 33: 1.0, 122: 1.0, 161: 1.0, 178: 1.0, 209: 1.0, 320: 1.0, 454: 1.0, 474: 1.0, 506: 1.0, 597: 1.0, 606: 1.0, 660: 1.0, 1438: 1.0, 1479: 1.0, 1483: 1.0, 1513: 1.0, 3499: 1.0, 7127: 1.0, 7858: 1.0, 8435: 1.0, 10784: 1.0, 14309: 1.0, 15131: 1.0, 15314: 1.0, 15746: 1.0, 16454: 1.0}))]

In [13]:
# feature data struct tester
num_feats = vectorizedRDD.take(1)[0][1].size
print("Number of total expanded features:", num_feats)

Number of total expanded features: 18545


In [14]:
print("Percent of data in the positive class:")
vectorizedRDD.map(lambda x: x[0]).mean()

Percent of data in the positive class:


0.254003558718861

## Develop Probability and Gradient Estimation Function

In [64]:
# initialize weights
np.random.seed(24)

b_br = sc.broadcast(0.0)
w_br = sc.broadcast(np.random.normal(0.0, 0.02, num_feats))
k_br = sc.broadcast(2)
V_br = sc.broadcast(np.random.normal(0.0, 0.02, (k_br.value, num_feats)))

# tester
#V_br.value[1][[1,2,1000]]
print(type(w_br.value))
print(type(V_br.value))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [16]:
import numpy as np
from pyspark.ml.feature import CountVectorizer
from scipy.sparse import csr_matrix

def predict_grad(pair):
    """
        Compute the predicted probability AND return the gradient (?)
        Args:
            pair - records are in (label, sparse feature set) format
        Broadcast:
            b - bias term (scalar)
            w - linear weight vector (array)
            k - number of factors (def=2)
            V - factor matrix of size (d dimensions, k=2 factors)
        Returns:
            predRDD - pair of ([label, predicted probability], [set of weight vectors in csr_matrix format])
    """
    
    label = pair[0]
    feats = pair[1]
    
    # start with linear weight dot product
    linear_sum = np.dot(w_br.value[feats.indices], feats.values)

    # factor matrix interaction sum
    factor_sum = 0.0
    lh_factor = [0.0]*k_br.value
    rh_factor = [0.0]*k_br.value
    
    for f in range(0, k_br.value):
        lh_factor[f] = np.dot(V_br.value[f][feats.indices], feats.values)  #KEY--this is used in v_grad matrix below
        rh_factor[f] = np.dot(V_br.value[f][feats.indices]**2, feats.values**2)
        factor_sum += (lh_factor[f]**2 - rh_factor[f])
    factor_sum = 0.5 * factor_sum
    
    pre_prob = b_br.value + linear_sum + factor_sum
    
    prob = 1.0 / (1 + np.exp(-pre_prob))  #logit transformation
    
    #compute Gradients
    b_grad = prob - label
    
    w_grad = csr_matrix((b_grad*feats.values, feats.indices, np.array([0, feats.indices.size])), (1, w_br.value.shape[0]))
    #print(w_grad[(0,5)])
    
    v_grad = csr_matrix((V_br.value.shape[0], V_br.value.shape[1]))
    for f in range(0, k_br.value):  # WORKING
        for i in feats.indices:
            i = int(i)
            v_grad[(f,i)] = b_grad * (feats[i]*lh_factor[f] - V_br.value[f][i]*(feats[i]**2))
    #print(type(v_grad))
    
    
    return ([label, prob], [b_grad, w_grad, v_grad])

In [17]:
test_csr = csr_matrix(np.array([0,1,0,2,0,6]).reshape(2,3))
test_csr[(1,1)] = 19
print(test_csr)

  (0, 1)	1
  (1, 0)	2
  (1, 1)	19
  (1, 2)	6


/opt/anaconda/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [18]:
# toy example
tester_grads = predict_grad(vectorizedRDD.take(1)[0])
sparse_mtx = tester_grads[1][2]
sparse_mtx[(0,5)]

/opt/anaconda/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


0.11675047804594206

In [38]:
predRDD = vectorizedRDD.map(predict_grad).cache()
test_pred = predRDD.take(1)
test_pred[0][1][1][(0,15746)]

0.46007271393366395

## Get log-loss with regularization on the sample set

In [20]:
def logLoss(pair):
    """parallelize log loss"""
    y = pair[0][0]
    
    eps = 1.0e-16
    if pair[0][1] == 0:
        y_hat = eps
    elif pair[0][1] == 1:
        y_hat = 1-eps
    else:
        y_hat = pair[0][1]
    
    return -(y * np.log(y_hat) + (1-y) * np.log(1-y_hat))


In [21]:
useReg = True
regParam = .001

sample_loss = predRDD.map(logLoss).mean() + int(useReg)*(regParam/2)*(np.linalg.norm(w_br.value)**2 + np.linalg.norm(V_br.value)**2)
sample_loss

0.7125966711201799

# Update weight vectors by average gradients with regularization (optional)

In [60]:
learningRate = 0.1

############## gradient calculation ##############
# calculate average gradient for b
bGrad = predRDD.map(lambda x: x[1][0]).mean()

# calculate average gradient for w
N = predRDD.count()
wGrad = (1/N) * predRDD.map(lambda x: x[1][1]).reduce(lambda a,b: a+b) 
print("wGrad shape:", wGrad.shape)
print(type(wGrad))

# calculate average gradient for V
vGrad = (1/N) * predRDD.map(lambda x: x[1][2]).reduce(lambda a,b: a+b)
print("vGrad shape:", vGrad.shape)
print(type(vGrad))

############## update weights ##############
# first, unpersist broadcasts
b_br.unpersist()
w_br.unpersist()
V_br.unpersist()

print("w_br shape:", w_br.value.shape)
print(type(w_br.value))

# update
b_br = sc.broadcast(b_br.value - learningRate * bGrad)
w_br = sc.broadcast(w_br.value - learningRate * wGrad.toarray())
V_br = sc.broadcast(V_br.value - learningRate * vGrad.toarray())

wGrad shape: (1, 18545)
<class 'scipy.sparse.csr.csr_matrix'>
vGrad shape: (2, 18545)
<class 'scipy.sparse.csr.csr_matrix'>
w_br shape: (1, 18545)
<class 'numpy.ndarray'>


In [66]:
print("Bias:", b_br.value)
print("W shape:", type(w_br.value))
print("V shape:", type(V_br.value))

print("W e.g.:", w_br.value[10])
print("V e.g.:", V_br.value[0][10])

Bias: 0.0
W shape: <class 'numpy.ndarray'>
V shape: <class 'numpy.ndarray'>
W e.g.: 0.013576095980501269
V e.g.: 0.0015767034959354965


# Iterate over Model

In [71]:
np.random.seed(24)

b_br = sc.broadcast(0.0)
w_br = sc.broadcast(np.random.normal(0.0, 0.02, num_feats))
k_br = sc.broadcast(2)
V_br = sc.broadcast(np.random.normal(0.0, 0.02, (k_br.value, num_feats)))


nIter = 2
learningRate = 0.1
useReg = False
regParam = .001
losses = []

for i in range(nIter):
    predRDD = vectorizedRDD.map(predict_grad).cache()
    loss = predRDD.map(logLoss).mean() #+ int(useReg)*(regParam/2)*(np.linalg.norm(w_br.value)**2 + np.linalg.norm(V_br.value)**2)
    losses.append(loss)
    print(i, "Current log-loss:", loss)
    
    # calculate average gradient for b
    bGrad = predRDD.map(lambda x: x[1][0]).mean()
    print("Bias:", bGrad)

    # calculate average gradient for w
    N = predRDD.count()
    wGrad = (1/N) * predRDD.map(lambda x: x[1][1]).reduce(lambda a,b: a+b)
    print("wGrad shape:", wGrad.shape)
    print(type(wGrad))

    # calculate average gradient for V
    vGrad = (1/N) * predRDD.map(lambda x: x[1][2]).reduce(lambda a,b: a+b)
    print("vGrad shape:", vGrad.shape)
    print(type(vGrad))

    ############## update weights ##############
    # first, unpersist broadcasts
    #predRDD.unpersist()
    b_br.unpersist()
    w_br.unpersist()
    V_br.unpersist()

    # update
    b_br = sc.broadcast(b_br.value - learningRate * bGrad)
    w_br = sc.broadcast(w_br.value - learningRate * wGrad.toarray())  # add regularization terms here conditional on useReg
    V_br = sc.broadcast(V_br.value - learningRate * vGrad.toarray())  # add regularization terms here conditional on useReg
    
    print(i, "Bias:", b_br.value)
    print(i, "W shape:", w_br.value.shape)
    print(i, "W shape:", type(w_br.value))
    print(i, "V shape:", V_br.value.shape)
    print(i, "V shape:", type(V_br.value))

0 Current log-loss: 0.701451869303434
Bias: 0.25193110161607585
wGrad shape: (1, 18545)
<class 'scipy.sparse.csr.csr_matrix'>
vGrad shape: (2, 18545)
<class 'scipy.sparse.csr.csr_matrix'>
0 Bias: -0.025193110161607586
0 W shape: (1, 18545)
0 W shape: <class 'numpy.ndarray'>
0 V shape: (2, 18545)
0 V shape: <class 'numpy.ndarray'>


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 94.0 failed 1 times, most recent failure: Lost task 0.0 in stage 94.0 (TID 22673, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-16-d6bfa833f48e>", line 23, in predict_grad
IndexError: index 1 is out of bounds for axis 0 with size 1


In [ ]:
losses